In [234]:
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import yfinance as yf
import plotly.graph_objs as go
from finta import TA

In [62]:
# Create a function to pull in data using Yahoo finance API
def asset_df(*x):
    
    data = yf.download(tickers=[*x], period = '90d', interval = '1d')
    df = data['Close'].dropna()
    return df

In [220]:
# Create df with variables using function
# Dow = ^DJI
# SP500 = ^GSPC
# 30-Year TSY = ^TYX
# Crypto pairs 'CRYPTOTICKER-USD'

df = asset_df('^TYX','BTC-USD','^DJI','^GSPC')
df

[*********************100%***********************]  4 of 4 completed


,BTC-USD,^DJI,^GSPC,^TYX
Date,,,,
2021-09-13,44963.074219,34869.628906,4468.729980,1.905
2021-09-14,47092.492188,34577.570312,4443.049805,1.850
2021-09-15,48176.347656,34814.390625,4480.700195,1.869
2021-09-16,47783.359375,34751.320312,4473.750000,1.881
2021-09-17,47267.519531,34584.878906,4432.990234,1.909
...,...,...,...,...
2021-12-03,53598.246094,34580.078125,4538.430176,1.678
2021-12-06,50582.625000,35227.031250,4591.669922,1.757
2021-12-07,50700.085938,35719.429688,4686.750000,1.796


In [221]:
# data = yf.download(tickers=['BTC-USD','ETH-USD'], period = '90d', interval = '1d')
# data

In [222]:
# Create df with yahoo finance close prices and a df to convert to % change
# crypto_price_df = data['Close']

# crypto_price_chg_df = crypto_price_df.pct_change()
# crypto_price_chg_df.dropna()

In [223]:

# # Graph Correlation between BTC & ETH
# crypto_corr = crypto_price_df.corr()
# crypto_corr_plot = crypto_corr.hvplot.heatmap()
# crypto_corr_plot

# Create function to run correlation matrix on any asset
def asset_corr(df):
    pct_chg_df = df.pct_change()
    df_corr = pct_chg_df.corr()
    df_corr_plot = df_corr.hvplot.heatmap()
    return df_corr_plot
    

In [224]:
asset_corr(df)

:HeatMap   [columns,index]   (value)

In [225]:
# Create chart with dropdown by crypto 
hist_crypto_prices = crypto_price_df.hvplot.line(x = 'Date', shared_axes = False)
hist_crypto_prices

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [210]:
# # plotly figure setup
# fig = go.Figure()

# # one trace for each df column
# for col in crypto_price_df.columns:
#     fig.add_trace(go.Scatter(x=crypto_price_df.index, y=crypto_price_df[col].values,
#                              name = col,
#                              mode = 'lines')
#                  )

# # one button for each df column
# updatemenu= []
# buttons=[]
# for col in crypto_price_df.columns:
#     buttons.append(dict(method='restyle',
#                         label=col,
#                         args=[{'y':[crypto_price_df[col].values]}])
#                   )

In [211]:
# # some adjustments to the updatemenu
# updatemenu=[]
# your_menu=dict()
# updatemenu.append(your_menu)
# updatemenu[0]['buttons']=buttons
# updatemenu[0]['direction']='down'
# updatemenu[0]['showactive']=True

# # update layout and show figure
# fig.update_layout(updatemenus=updatemenu)
# fig.show()

In [212]:
# # Read in .csv file using bloomberg static data
# df = pd.read_csv('data.csv', index_col = 'Date')

# #DROP S&P500 AND TSY COLUMNS FOR CUPCAKE. FEED DATA IN WHEN WE GET
# df.drop(columns = ['ETH','SPY','TSY'],inplace=True)

# #DROP NULLS
# df.dropna()

In [226]:
df.head()

,BTC-USD,^DJI,^GSPC,^TYX
Date,,,,
2021-09-13,44963.074219,34869.628906,4468.729980,1.905
2021-09-14,47092.492188,34577.570312,4443.049805,1.850
2021-09-15,48176.347656,34814.390625,4480.700195,1.869
2021-09-16,47783.359375,34751.320312,4473.750000,1.881
2021-09-17,47267.519531,34584.878906,4432.990234,1.909


In [227]:
# Convert df to % change for analysis
pct_chg_df = df.pct_change()
pct_chg_df = pct_chg_df.dropna()

# RF Model
- model r2 score was negative using Gold, Oil, SP500, DJI
- maybe we should try and predict the prices of a crypto asset given price movements in other cryptos

In [228]:
# Define features set
X = pct_chg_df.copy()
X.drop("BTC-USD", axis=1, inplace=True)
X.head()

,^DJI,^GSPC,^TYX
Date,,,
2021-09-14,-0.008376,-0.005747,-0.028871
2021-09-15,0.006849,0.008474,0.010270
2021-09-16,-0.001812,-0.001551,0.006421
2021-09-17,-0.004789,-0.009111,0.014886
2021-09-20,-0.017765,-0.016977,-0.033002


In [230]:
# Define target vector
y = pct_chg_df["BTC-USD"].values.reshape(-1, 1)
y[:5]

array([[ 0.04735926],
       [ 0.02301546],
       [-0.00815729],
       [-0.01079539],
       [-0.09358898]])

In [231]:
# Create function for RF Model

def RF(X,y):
    
    #split data into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
    
    #initiate, fit, and train rf_model
    rf_model = RandomForestRegressor(n_estimators=500, random_state=42)
    rf_model = rf_model.fit(X_train, y_train)
    
    #make predictions for target using X_test
    predictions = rf_model.predict(X_test)
    
    return rf_model.score(X_test, y_test)

In [232]:
RF(X,y)

/Users/jonyamaoka/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


-0.4301925165602596

In [186]:
# Splitting into Train and Test sets

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [187]:
# rf_model = RandomForestRegressor(n_estimators=500, random_state=42)

In [188]:
# Fit the model
# rf_model = rf_model.fit(X_train, y_train)

/Users/jonyamaoka/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [189]:
# Making predictions using the testing data
# predictions = rf_model.predict(X_test)

In [190]:
# #Compare predictions vs actual
# y_test_df = pd.DataFrame(y_test)
# y_test_df['predict']=predictions
# y_test_df.hvplot()

In [202]:
# Get the feature importance array
importances = rf_model.feature_importances_

In [203]:
# List the top 10 most important features
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
importances_sorted[:10]

[(0.6182338050234543, 'COIN'),
 (0.09825107540422884, 'GC=F'),
 (0.08985778639768094, 'GME'),
 (0.08120592520826976, '^DJI'),
 (0.05631319421260322, 'CL=F'),
 (0.05613821375376278, '^GSPC')]

In [193]:
# rf_model.score(X_test, y_test)

0.04271679713311516

# ALGO TRADING FOR INDIVIDUAL SECURITY

In [238]:
# Create a new clean copy of the signals_df DataFrame
bb_signals_df = yf.download(tickers='BTC-USD', period = '90d', interval = '1d')

# Review the DataFrame
bb_signals_df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-09-13,46057.214844,46598.679688,43591.320312,44963.074219,44963.074219,40969943253
2021-09-14,44960.050781,47218.125000,44752.332031,47092.492188,47092.492188,38652152880
2021-09-15,47098.000000,48450.468750,46773.328125,48176.347656,48176.347656,30484496466
2021-09-16,48158.906250,48486.828125,47079.558594,47783.359375,47783.359375,31764293754
2021-09-17,47771.003906,48160.921875,46832.523438,47267.519531,47267.519531,28727713711


In [243]:
def bollBands(ticker):
    # Create a new clean copy of the signals_df DataFrame
    bb_signals_df = yf.download(tickers = ticker, period = '90d', interval = '1d')
    
    # Determine the Bollinger Bands for the Dataset
    bbands_df = TA.BBANDS(bb_signals_df)
    
    # Concatenate the Bollinger Bands to the DataFrame
    bb_signals_df = pd.concat([bb_signals_df, bbands_df], axis=1)
    
    # Visualize close price for the investment
    security_close = bb_signals_df[["Close"]].hvplot(
        line_color='lightgray',
        ylabel='Price in $',
        width=1000,
        height=400
    )

    bb_upper = bb_signals_df[["BB_UPPER"]].hvplot(
        line_color='purple',
        ylabel='Price in $',
        width=1000,
        height=400
    )


    bb_middle = bb_signals_df[["BB_MIDDLE"]].hvplot(
        line_color='orange',
        ylabel='Price in $',
        width=1000,
        height=400
    )

    bb_lower = bb_signals_df[["BB_LOWER"]].hvplot(
        line_color='blue',
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Overlay plots
    bbands_plot = security_close * bb_upper * bb_middle * bb_lower
    return bbands_plot

In [245]:
bollBands('TSLA')

[*********************100%***********************]  1 of 1 completed


:Overlay
   .Curve.I   :Curve   [Date]   (Close)
   .Curve.II  :Curve   [Date]   (BB_UPPER)
   .Curve.III :Curve   [Date]   (BB_MIDDLE)
   .Curve.IV  :Curve   [Date]   (BB_LOWER)

In [239]:
# Determine the Bollinger Bands for the Dataset
bbands_df = TA.BBANDS(bb_signals_df)

# Review the DataFrame
bbands_df.tail()

,BB_UPPER,BB_MIDDLE,BB_LOWER
Date,,,
2021-12-07,61790.367921,55425.315039,49060.262157
2021-12-08,61788.542231,55103.448047,48418.353863
2021-12-09,61878.513995,54581.075195,47283.636396
2021-12-10,61537.620083,53958.380664,46379.141245
2021-12-11,61016.934283,53467.562500,45918.190717


In [240]:
# Concatenate the Bollinger Bands to the DataFrame
bb_signals_df = pd.concat([bb_signals_df, bbands_df], axis=1)

# Review the DataFrame
bb_signals_df.tail()

,Open,High,Low,Close,Adj Close,Volume,BB_UPPER,BB_MIDDLE,BB_LOWER
Date,,,,,,,,,
2021-12-07,50581.828125,51934.781250,50175.808594,50700.085938,50700.085938,33676814852,61790.367921,55425.315039,49060.262157
2021-12-08,50667.648438,51171.375000,48765.988281,50504.796875,50504.796875,28479699446,61788.542231,55103.448047,48418.353863
2021-12-09,50450.082031,50797.164062,47358.351562,47672.121094,47672.121094,29603577251,61878.513995,54581.075195,47283.636396
2021-12-10,47642.144531,50015.253906,47023.699219,47243.304688,47243.304688,30966005122,61537.620083,53958.380664,46379.141245
2021-12-11,47339.375000,49226.472656,46942.347656,48914.113281,48914.113281,25972500480,61016.934283,53467.562500,45918.190717


In [242]:
# Visualize close price for the investment
security_close = bb_signals_df[["Close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_upper = bb_signals_df[["BB_UPPER"]].hvplot(
    line_color='purple',
    ylabel='Price in $',
    width=1000,
    height=400
)


bb_middle = bb_signals_df[["BB_MIDDLE"]].hvplot(
    line_color='orange',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_lower = bb_signals_df[["BB_LOWER"]].hvplot(
    line_color='blue',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
bbands_plot = security_close * bb_upper * bb_middle * bb_lower
bbands_plot

:Overlay
   .Curve.I   :Curve   [Date]   (Close)
   .Curve.II  :Curve   [Date]   (BB_UPPER)
   .Curve.III :Curve   [Date]   (BB_MIDDLE)
   .Curve.IV  :Curve   [Date]   (BB_LOWER)